In [3]:
import pandas as pd
import requests
import json

In [2]:
from API_keys import Aerodata_API_key as key

In [6]:
latitudes= [52.5167, 53.55]
longitudes =[13.3833, 10.0]

In [4]:
def Airport_ICAO(latitudes, longitudes):
    from API_keys import Aerodata_API_key as key
    Airport_Info_df = []

    for index, value in enumerate(latitudes):

        url = f"https://aerodatabox.p.rapidapi.com/airports/search/location/{value}/{longitudes[index]}/km/50/16"

        querystring = {"withFlightInfoOnly":"true"}

        headers = {
      "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
      "X-RapidAPI-Key": key
    }

        response = requests.request("GET", url, headers=headers, params=querystring)

        Airport_Info_df.append(pd.json_normalize(response.json()['items']))

    return pd.concat(Airport_Info_df, ignore_index=True)

In [16]:
Airport_df =Airport_ICAO(latitudes, longitudes).pipe(reshape_rename)
Airport_df

,icao,city_code,city_name,airport_latitude,airport_longitude
0,EDDB,BER,Berlin,52.35139,13.493889
1,EDDH,HAM,Hamburg,53.63040,9.988229


In [15]:
def reshape_rename(df):
    df = df.rename(columns={'municipalityName':'city_name','iata':'city_code', 'location.lat':'airport_latitude', 'location.lon':'airport_longitude'})
    df = df.drop(["name", "shortName", "countryCode"], axis=1)
    return df

In [17]:
from aws_con import con

In [18]:
Airport_df.to_sql('Airports',
              if_exists='append',
              con=con,
              index=False)

2